### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with Numeric digits in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [1]:
import pandas as pd 
import numpy as np

In [3]:
Energy=pd.read_excel("assets/Energy Indicators.xls")

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

In [4]:
pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 704.9 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)

In [3]:
Energy.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Energy Supply,Energy Supply per capita,Renewable Electricity Production
0,NaN,NaN,NaN,Petajoules,Gigajoules,%
1,NaN,Afghanistan,Afghanistan,321,10,78.66928
2,NaN,Albania,Albania,102,35,100
3,NaN,Algeria,Algeria,1959,51,0.55101
4,NaN,American Samoa,American Samoa,...,...,0.641026


In [33]:
len(Energy)

228

In [34]:
Energy.loc[227]

Unnamed: 0                               NaN
Unnamed: 1                          Zimbabwe
Unnamed: 2                          Zimbabwe
Energy Supply                            480
Energy Supply per capita                  32
Renewable Electricity Production    52.53612
Name: 227, dtype: object

In [35]:
Energy.iloc[0]

Unnamed: 0                                 NaN
Unnamed: 1                                 NaN
Unnamed: 2                                 NaN
Energy Supply                       Petajoules
Energy Supply per capita            Gigajoules
Renewable Electricity Production             %
Name: 0, dtype: object

In [45]:
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)

In [12]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=17)
Energy=Energy.drop([Energy.columns[0],Energy.columns[1]],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Petajoules':'Energy Supply','Gigajoules':'Energy Supply per Capita','%':'% Renewable'})
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})
Energy.head()


,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650


In [9]:

Energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321,10,78.669280
1,Albania,102,35,100.000000
2,Algeria,1959,51,0.551010
3,American Samoa,...,...,0.641026
4,Andorra,9,121,88.695650


In [10]:
Energy.columns

Index(['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'], dtype='object')

In [48]:
Energy=Energy.drop(0)

In [49]:
Energy.head()

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,321,10,78.66928
2,Albania,102,35,100
3,Algeria,1959,51,0.55101
4,American Samoa,...,...,0.641026
5,Andorra,9,121,88.69565


In [50]:
Energy=Energy.set_index('Country')

In [51]:
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,321,10,78.66928
Albania,102,35,100
Algeria,1959,51,0.55101
American Samoa,...,...,0.641026
Andorra,9,121,88.69565


In [ ]:
Energy.replace("...",np.nan,inplace=True)

In [52]:
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']

In [53]:
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,321000000,10,78.66928
Albania,102000000,35,100
Algeria,1959000000,51,0.55101
American Samoa,.................................................,...,0.641026
Andorra,9000000,121,88.69565


In [54]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy=Energy.set_index('Country')
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,3.210000e+08,10.0,78.66928
Albania,1.020000e+08,35.0,100
Algeria,1.959000e+09,51.0,0.55101
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.69565


In [71]:

Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()




,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,3.210000e+08,10.0,78.669280
2,Albania,1.020000e+08,35.0,100.000000
3,Algeria,1.959000e+09,51.0,0.551010
4,American Samoa,NaN,NaN,0.641026
5,Andorra,9.000000e+06,121.0,88.695650


In [72]:
Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

In [77]:
Energy=Energy.set_index('Country')


In [87]:
Energy.loc['South Korea']

Energy Supply               1.100700e+10
Energy Supply per capita    2.210000e+02
% Renewable                 2.279353e+00
Name: South Korea, dtype: float64

In [81]:
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,3.210000e+08,10.0,78.669280
Albania,1.020000e+08,35.0,100.000000
Algeria,1.959000e+09,51.0,0.551010
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.695650


In [89]:
print(Energy.index.tolist())

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia1', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China2', 'China, Hong Kong Special Administrative Region3', 'China, Macao Special Administrative Region4', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark5', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador'

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()

Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

Energy=Energy.set_index('Country')
Energy.rename(index=lambda x: x.replace(r'\([^)]*\)', '').strip(), inplace=True)

In [3]:
print(Energy.index.tolist())

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia1', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China2', 'China, Hong Kong Special Administrative Region3', 'China, Macao Special Administrative Region4', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark5', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador'

In [16]:
countries= Energy.index.tolist()

In [17]:
countries = [re.sub(r'\([^)]*\)', '', country).strip() for country in countries]

In [7]:
import re

In [9]:
print(countries)


['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia1', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China2', 'China, Hong Kong Special Administrative Region3', 'China, Macao Special Administrative Region4', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark5', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',

In [18]:
countries = [re.sub(r'\d+', '', country).strip() for country in countries]

In [19]:
print(countries)

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'China, Hong Kong Special Administrative Region', 'China, Macao Special Administrative Region', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equ

In [ ]:
Energy.index=countries

In [20]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()

Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

Energy=Energy.set_index('Country')
Energy.rename(index=lambda x: x.replace(r'\([^)]*\)', '').strip(), inplace=True)

countries = [re.sub(r'\([^)]*\)', '', country).strip() for country in countries]
countries = [re.sub(r'\d+', '', country).strip() for country in countries]
Energy.index=countries

In [22]:
Energy.loc['Italy']

Energy Supply               6.530000e+09
Energy Supply per capita    1.090000e+02
% Renewable                 3.366723e+01
Name: Italy, dtype: float64